Simple demo example for learning the DLS model from simulated data:

In [1]:
from __future__ import division
from helper import *
from embedding import *
from hawkes_embedding import HawkesEmbedding

Simple simulated dataset:

In [3]:
rand.seed(123)

B = 4  # Number of basis kernels
N = 5  # Number of nodes
D = 2  # Number of latent dimensions
T = 100  # End time

# Simulate parameters 
gamma = rand.uniform(low=0, high=.2)  # Base rate
beta = rand.uniform(low=0, high=.1)  # Jump size
xvec = rand.uniform(size=(N, B, D))  # Reciprocal latent features
zvec = rand.uniform(size=(N, D))   # Homophily latent features

# Initialize HawkesEmbedding and set parameters
hawkes = HawkesEmbedding(num_nodes=N, events=list(), end_time=T, num_dims=D)
assert_equal(B, hawkes.B)
hawkes.set_params(end_time=T, gamma=np.zeros((N, N)) + gamma,
                  beta=np.zeros((N, N)) + beta, xvec=xvec, zvec=zvec)
xi_true = hawkes.update_xi()

# Simulate Hawkes events
events = hawkes.simulate()

Simulated 32 events for node 0 in [0, 100.00).
Simulated 24 events for node 1 in [0, 100.00).
Simulated 30 events for node 2 in [0, 100.00).
Simulated 9 events for node 3 in [0, 100.00).
Simulated 10 events for node 4 in [0, 100.00).
Simulated 46 events for (0, 1) node pair in [0, 100.00).
Simulated 34 events for (0, 2) node pair in [0, 100.00).
Simulated 23 events for (0, 3) node pair in [0, 100.00).
Simulated 20 events for (0, 4) node pair in [0, 100.00).
Simulated 14 events for (1, 2) node pair in [0, 100.00).
Simulated 27 events for (1, 3) node pair in [0, 100.00).
Simulated 10 events for (1, 4) node pair in [0, 100.00).
Simulated 41 events for (2, 3) node pair in [0, 100.00).
Simulated 37 events for (2, 4) node pair in [0, 100.00).
Simulated 35 events for (3, 4) node pair in [0, 100.00).


In [4]:
events

[(1, 3, 10.422390248122673),
 (1, 3, 11.470563734711277),
 (1, 3, 16.208594173679401),
 (1, 3, 16.353820043213119),
 (1, 3, 16.688176016716767),
 (1, 3, 24.307766729927785),
 (1, 3, 25.009929863858396),
 (1, 3, 26.092885087486415),
 (1, 3, 27.695558025107534),
 (1, 3, 52.781503899649003),
 (1, 3, 58.018373022650756),
 (1, 3, 58.429444061517856),
 (1, 3, 77.94340866393047),
 (1, 3, 85.117077281691962),
 (3, 0, 9.0201987888422881),
 (3, 0, 23.048895764949712),
 (3, 0, 24.519551986133873),
 (3, 0, 27.357040812391745),
 (3, 0, 48.335623471234108),
 (3, 0, 48.795951114448449),
 (3, 0, 49.3430453820081),
 (3, 0, 66.503104061204297),
 (3, 0, 93.302659445116944),
 (3, 0, 99.788005614900896),
 (2, 1, 9.8766472173915449),
 (2, 1, 21.523680938321831),
 (2, 1, 34.661411216919809),
 (2, 1, 43.07170867774704),
 (2, 1, 56.653367642857468),
 (2, 1, 67.839832512220013),
 (2, 1, 81.92136137881414),
 (2, 1, 82.525116084088467),
 (2, 1, 89.856135090643761),
 (0, 3, 28.71205358080034),
 (0, 3, 34.207394042

Learn the DLS model parameters:

In [9]:
hawkes_test = HawkesEmbedding(num_nodes=N, events=events,
                              end_time=T, num_dims=D)
hawkes_test.C_beta = 0.
hawkes_test.C_zvec = 1.  # Regularization on the magnitude of zvec's
hawkes_test.C_xvec1 = 1.  # Regularization on the magnitude of xvec's
hawkes_test.C_xvec2 = 1.  # Enforce similarity between the different basis-kernel dimensions for each node

In [12]:
hawkes_test.loglik(gamma=1, beta=beta, xvec=xvec, zvec=zvec)

-827.24583559106577

In [16]:
gamma_mle, beta_mle, xvec_mle, zvec_mle = hawkes_test.mle()
# gamma_mle, beta_mle, xvec_mle = hawkes_test.mle(method='coord-ascent')

1 [ 0.085  0.431  0.676  0.615  0.99   0.661  0.618  0.928  0.903  0.835
  0.407  0.698  0.956  0.24   0.487  0.758  0.598  0.784  0.264  0.131
  0.471  0.665  0.598  0.611  0.092  0.491  0.202  0.711  0.558  0.601
  0.596  0.305  0.083  0.412  0.298  0.57   0.223  0.223  0.324  0.172
  0.402  0.825  0.852  0.108  0.331  0.613  0.227  0.919  0.707  0.22
  0.107  0.074] [ 1384.161]
2 [ 0.047  0.3    0.676  0.615  0.991  0.662  0.621  0.938  0.909  0.836
  0.407  0.698  0.957  0.238  0.485  0.76   0.598  0.785  0.264  0.131
  0.47   0.666  0.6    0.614  0.088  0.489  0.202  0.711  0.558  0.601
  0.599  0.298  0.079  0.409  0.298  0.57   0.222  0.222  0.318  0.164
  0.403  0.827  0.853  0.108  0.331  0.614  0.226  0.921  0.707  0.22
  0.106  0.073] [ 1124.304]
3 [ 0.029  0.188  0.676  0.615  0.991  0.662  0.622  0.942  0.911  0.837
  0.407  0.698  0.957  0.238  0.484  0.761  0.599  0.785  0.264  0.131
  0.47   0.666  0.601  0.616  0.087  0.488  0.202  0.711  0.558  0.602
  0.601  0.294  0

  0.011  0.027] [ 823.652]
31 [ 0.163  0.023  0.659  0.609  0.955  0.672  0.602  0.986  0.892  0.832
  0.409  0.684  0.96   0.215  0.479  1.012  0.761  0.829  0.264  0.151
  0.419  0.625  0.529  0.426  0.11   0.496  0.214  0.676  0.508  0.567
  0.591  0.113 -0.031  0.321  0.299  0.554  0.25   0.288  0.361  0.186
  0.327  0.772  0.661  0.56   0.852  0.724  0.202  0.193  0.334  0.291
  0.002  0.018] [ 823.6]
32 [ 0.163  0.024  0.659  0.609  0.953  0.672  0.601  0.985  0.891  0.832
  0.409  0.683  0.959  0.214  0.479  1.017  0.764  0.829  0.264  0.152
  0.417  0.624  0.527  0.423  0.111  0.496  0.215  0.675  0.507  0.565
  0.59   0.108 -0.034  0.318  0.299  0.553  0.251  0.29   0.362  0.186
  0.325  0.77   0.658  0.573  0.856  0.718  0.207  0.179  0.328  0.297
 -0.005  0.012] [ 823.57]
33 [ 0.162  0.025  0.658  0.609  0.951  0.672  0.6    0.985  0.89   0.831
  0.409  0.683  0.959  0.215  0.479  1.02   0.766  0.829  0.264  0.153
  0.417  0.623  0.526  0.421  0.111  0.496  0.215  0.674  0.5

60 [ 0.182  0.037  0.506  0.56   0.511  0.662  0.413  0.868  0.723  0.764
  0.447  0.585  0.772  0.257  0.56   1.582  1.201  0.838  0.244  0.281
  0.15   0.391  0.231  0.109  0.179  0.402  0.274  0.352  0.3    0.234
  0.38  -0.614 -0.406 -0.145  0.3    0.42   0.397  0.567  0.393  0.268
  0.135  0.505  0.445  0.309  0.427  0.591  0.211 -0.322 -0.049  0.057
 -0.443 -0.122] [ 820.166]
61 [ 0.184  0.037  0.505  0.559  0.517  0.665  0.418  0.883  0.741  0.767
  0.448  0.588  0.777  0.255  0.559  1.552  1.179  0.837  0.242  0.277
  0.153  0.397  0.236  0.127  0.173  0.399  0.271  0.353  0.309  0.242
  0.381 -0.592 -0.396 -0.141  0.3    0.423  0.382  0.548  0.386  0.244
  0.132  0.509  0.469  0.296  0.412  0.614  0.23  -0.337 -0.039  0.067
 -0.459 -0.106] [ 820.089]
62 [ 0.19   0.036  0.494  0.555  0.49   0.667  0.409  0.884  0.74   0.765
  0.451  0.583  0.768  0.256  0.565  1.571  1.194  0.836  0.24   0.282
  0.137  0.385  0.22   0.113  0.172  0.39   0.272  0.332  0.301  0.224
  0.366 -0.628

 -0.599 -0.357] [ 818.948]
92 [ 0.196  0.046  0.286  0.45   0.091  0.806  0.38   1.014  0.906  0.802
  0.536  0.543  0.872  0.184  0.691  1.5    1.111  0.807  0.147  0.261
 -0.105  0.227  0.044 -0.24  -0.111  0.039  0.201 -0.035  0.233  0.028
  0.055 -0.933 -0.542 -0.679  0.263  0.319  0.082  0.356  0.114  0.187
  0.025  0.448  0.348 -0.026  0.118  0.517  0.082 -0.468 -0.344  0.002
 -0.632 -0.397] [ 818.933]
93 [ 0.195  0.046  0.294  0.453  0.111  0.803  0.385  1.016  0.909  0.803
  0.533  0.547  0.874  0.185  0.686  1.484  1.099  0.808  0.149  0.258
 -0.092  0.237  0.055 -0.22  -0.106  0.05   0.201 -0.019  0.241  0.041
  0.068 -0.907 -0.529 -0.656  0.264  0.325  0.084  0.351  0.119  0.181
  0.031  0.455  0.358 -0.013  0.131  0.524  0.092 -0.461 -0.327  0.014
 -0.621 -0.385] [ 818.916]
94 [ 0.195  0.046  0.298  0.455  0.12   0.802  0.388  1.017  0.91   0.803
  0.532  0.548  0.877  0.185  0.684  1.475  1.093  0.808  0.15   0.255
 -0.087  0.242  0.06  -0.212 -0.105  0.054  0.201 -0.012  

 -0.626 -0.435] [ 818.268]
122 [ 0.194  0.05   0.167  0.375 -0.118  0.766  0.259  0.758  0.725  0.738
  0.626  0.511  1.102  0.158  0.936  1.261  1.035  0.776 -0.004  0.063
 -0.258  0.063 -0.101 -0.238 -0.312 -0.194  0.076 -0.18   0.158 -0.059
 -0.129 -0.86  -0.345 -0.879  0.136  0.27  -0.148  0.175 -0.116  0.105
 -0.146  0.474  0.331  0.037  0.107  0.492  0.111 -0.485 -0.357  0.013
 -0.629 -0.438] [ 818.263]
123 [ 0.198  0.05   0.145  0.361 -0.151  0.754  0.25   0.743  0.701  0.728
  0.643  0.504  1.125  0.161  0.968  1.216  1.024  0.768 -0.029  0.032
 -0.283  0.034 -0.13  -0.238 -0.341 -0.227  0.058 -0.204  0.144 -0.08
 -0.159 -0.878 -0.319 -0.919  0.114  0.26  -0.178  0.152 -0.155  0.099
 -0.18   0.476  0.327  0.041  0.105  0.5    0.117 -0.49  -0.365  0.002
 -0.635 -0.444] [ 818.225]
124 [ 0.199  0.05   0.126  0.349 -0.177  0.739  0.242  0.727  0.676  0.719
  0.657  0.497  1.144  0.164  0.996  1.173  1.014  0.76  -0.052  0.003
 -0.304  0.007 -0.155 -0.232 -0.365 -0.253  0.042 -0.223

...

213 [ 0.195  0.101 -0.152  0.23  -0.407  0.528  0.179  0.979  0.425  0.621
  0.781  0.108  1.055  0.088  1.317  0.203  0.962  0.454 -0.349 -0.364
 -0.523 -0.347 -0.882 -0.498 -0.727 -0.541  0.066 -0.353  0.081 -0.286
  0.185 -1.155  0.123 -1.158 -0.394  0.31  -0.285 -0.044 -0.864  0.4   -0.836
  0.541  0.402  0.115  0.111  0.589  0.245 -0.373 -0.27  -0.001 -0.538
 -0.374] [ 817.051]
214 [ 0.195  0.103 -0.152  0.232 -0.407  0.533  0.179  0.983  0.423  0.62
  0.782  0.103  1.058  0.083  1.321  0.203  0.964  0.453 -0.35  -0.366
 -0.524 -0.346 -0.884 -0.505 -0.728 -0.542  0.068 -0.351  0.081 -0.286
  0.19  -1.153  0.125 -1.158 -0.396  0.313 -0.286 -0.045 -0.872  0.402
 -0.838  0.544  0.401  0.114  0.111  0.59   0.248 -0.373 -0.271 -0.002
 -0.539 -0.372] [ 817.048]
215 [ 0.194  0.106 -0.152  0.242 -0.41   0.554  0.179  0.998  0.418  0.62
  0.781  0.086  1.067  0.06   1.333  0.212  0.979  0.454 -0.35  -0.369
 -0.53  -0.341 -0.889 -0.527 -0.732 -0.543  0.078 -0.345  0.082 -0.289
  0.207 -1.15

243 [ 0.193  0.14  -0.175  0.337 -0.351  0.693  0.162  1.037  0.364  0.673
  0.849 -0.049  1.154 -0.101  1.474  0.292  1.036  0.435 -0.378 -0.457
 -0.659 -0.336 -0.934 -0.647 -0.807 -0.569  0.117 -0.317  0.115 -0.33
  0.326 -1.21   0.249 -1.249 -0.473  0.415 -0.322  0.004 -1.09   0.458
 -0.903  0.638  0.417  0.113  0.111  0.598  0.247 -0.353 -0.248 -0.004
 -0.533 -0.359] [ 816.961]
244 [ 0.193  0.138 -0.175  0.334 -0.345  0.685  0.163  1.032  0.365  0.677
  0.853 -0.041  1.148 -0.091  1.472  0.288  1.031  0.433 -0.379 -0.458
 -0.657 -0.338 -0.926 -0.64  -0.803 -0.571  0.112 -0.321  0.114 -0.328
  0.316 -1.209  0.25  -1.246 -0.473  0.414 -0.32   0.001 -1.086  0.459
 -0.905  0.637  0.419  0.114  0.112  0.601  0.25  -0.354 -0.249  0.    -0.53
 -0.359] [ 816.96]
245 [ 0.193  0.14  -0.176  0.335 -0.343  0.684  0.162  1.031  0.364  0.678
  0.855 -0.041  1.148 -0.091  1.473  0.285  1.029  0.431 -0.38  -0.46
 -0.658 -0.339 -0.926 -0.642 -0.804 -0.573  0.111 -0.321  0.115 -0.328
  0.317 -1.209 

 -0.526 -0.354] [ 816.95]
273 [ 0.193  0.146 -0.179  0.344 -0.325  0.721  0.158  1.012  0.351  0.706
  0.895 -0.018  1.148 -0.102  1.486  0.254  1.06   0.449 -0.382 -0.468
 -0.676 -0.338 -0.897 -0.604 -0.798 -0.592  0.102 -0.363  0.129 -0.334
  0.354 -1.23   0.269 -1.243 -0.489  0.45  -0.316 -0.005 -1.145  0.512
 -0.932  0.632  0.419  0.123  0.118  0.601  0.254 -0.349 -0.244 -0.001
 -0.526 -0.355] [ 816.95]
274 [ 0.192  0.147 -0.179  0.343 -0.326  0.723  0.158  1.01   0.351  0.706
  0.897 -0.015  1.149 -0.102  1.486  0.252  1.061  0.452 -0.381 -0.468
 -0.675 -0.338 -0.895 -0.599 -0.798 -0.592  0.102 -0.366  0.13  -0.334
  0.357 -1.233  0.27  -1.243 -0.489  0.452 -0.316 -0.005 -1.149  0.517
 -0.933  0.63   0.419  0.122  0.118  0.601  0.252 -0.348 -0.244 -0.002
 -0.525 -0.355] [ 816.949]
275 [ 0.192  0.146 -0.179  0.343 -0.326  0.722  0.158  1.01   0.351  0.705
  0.896 -0.016  1.149 -0.102  1.486  0.254  1.06   0.451 -0.381 -0.467
 -0.675 -0.338 -0.895 -0.6   -0.798 -0.591  0.102 -0.365 

 -0.528 -0.356] [ 816.942]
302 [ 0.191  0.153 -0.187  0.338 -0.322  0.746  0.154  0.977  0.345  0.727
  0.919 -0.003  1.165 -0.109  1.516  0.254  1.075  0.457 -0.358 -0.464
 -0.68  -0.34  -0.831 -0.537 -0.8   -0.594  0.108 -0.356  0.143 -0.327
  0.369 -1.289  0.303 -1.246 -0.499  0.465 -0.323  0.01  -1.224  0.576
 -0.94   0.637  0.411  0.124  0.115  0.595  0.245 -0.355 -0.243 -0.008
 -0.527 -0.356] [ 816.942]
303 [ 0.191  0.153 -0.187  0.338 -0.321  0.746  0.154  0.977  0.345  0.727
  0.92  -0.002  1.165 -0.109  1.516  0.254  1.076  0.458 -0.359 -0.464
 -0.68  -0.339 -0.832 -0.537 -0.8   -0.595  0.108 -0.357  0.143 -0.328
  0.37  -1.29   0.302 -1.246 -0.499  0.466 -0.322  0.011 -1.224  0.577
 -0.94   0.636  0.412  0.124  0.114  0.596  0.244 -0.355 -0.243 -0.008
 -0.527 -0.356] [ 816.942]
304 [ 0.191  0.153 -0.187  0.339 -0.321  0.747  0.153  0.976  0.345  0.728
  0.92  -0.002  1.165 -0.109  1.516  0.254  1.076  0.459 -0.359 -0.464
 -0.681 -0.339 -0.832 -0.537 -0.8   -0.595  0.108 -0.35

330 [ 0.191  0.154 -0.186  0.336 -0.325  0.753  0.146  0.968  0.345  0.728
  0.915 -0.006  1.168 -0.103  1.518  0.247  1.08   0.459 -0.344 -0.452
 -0.68  -0.348 -0.813 -0.514 -0.799 -0.591  0.118 -0.348  0.146 -0.325
  0.377 -1.296  0.304 -1.248 -0.509  0.463 -0.317  0.016 -1.235  0.588
 -0.937  0.644  0.412  0.127  0.114  0.594  0.242 -0.357 -0.243 -0.011
 -0.529 -0.356] [ 816.941]
331 [ 0.191  0.154 -0.186  0.336 -0.325  0.753  0.146  0.968  0.345  0.728
  0.915 -0.006  1.168 -0.103  1.518  0.247  1.08   0.459 -0.344 -0.452
 -0.68  -0.348 -0.813 -0.514 -0.799 -0.591  0.118 -0.348  0.146 -0.325
  0.377 -1.296  0.304 -1.248 -0.509  0.463 -0.317  0.016 -1.235  0.587
 -0.937  0.644  0.412  0.127  0.114  0.594  0.242 -0.357 -0.243 -0.011
 -0.529 -0.356] [ 816.941]
332 [ 0.191  0.154 -0.186  0.336 -0.325  0.753  0.146  0.968  0.345  0.728
  0.915 -0.007  1.168 -0.103  1.518  0.247  1.08   0.459 -0.344 -0.452
 -0.68  -0.348 -0.812 -0.514 -0.799 -0.591  0.118 -0.348  0.146 -0.325
  0.377 -1.

These are the learned parameter values:

In [18]:
gamma_mle
beta_mle

array([ 0.191])

array([ 0.154])

In [20]:
zvec_mle

array([[ 0.413,  0.127],
       [ 0.114,  0.595],
       [ 0.243, -0.356],
       [-0.243, -0.01 ],
       [-0.528, -0.356]])

In [22]:
xvec_mle

array([[[-0.186,  0.337],
        [-0.324,  0.75 ],
        [ 0.146,  0.97 ],
        [ 0.346,  0.727]],

       [[ 0.914, -0.008],
        [ 1.168, -0.102],
        [ 1.517,  0.249],
        [ 1.078,  0.458]],

       [[-0.342, -0.451],
        [-0.679, -0.348],
        [-0.813, -0.516],
        [-0.798, -0.59 ]],

       [[ 0.119, -0.349],
        [ 0.146, -0.324],
        [ 0.376, -1.297],
        [ 0.303, -1.248]],

       [[-0.511,  0.463],
        [-0.319,  0.016],
        [-1.232,  0.586],
        [-0.936,  0.645]]])

In [23]:
hawkes_test.loglik(gamma_mle, beta_mle, xvec_mle, zvec_mle)

array([-812.984])

In [29]:
hawkes_test.gamma[:] = gamma_mle
hawkes_test.beta[:] = beta_mle
hawkes_test.xvec[:] = xvec_mle
hawkes_test.zvec[:] = zvec_mle
xi = hawkes_test.update_xi()
xi[:, :, 1]

array([[ 1.   ,  0.052,  0.264,  0.253,  0.584],
       [ 0.052,  1.   ,  0.031,  0.335,  0.108],
       [ 0.264,  0.031,  1.   ,  0.506,  0.77 ],
       [ 0.253,  0.335,  0.506,  1.   ,  0.717],
       [ 0.584,  0.108,  0.77 ,  0.717,  1.   ]])

View the learned node-similarity matrices:

In [31]:
hawkes_test.dist_mat

array([[[ 1.   ,  0.734,  0.769,  0.638,  0.327],
        [ 0.734,  1.   ,  0.398,  0.611,  0.268],
        [ 0.769,  0.398,  1.   ,  0.701,  0.552],
        [ 0.638,  0.611,  0.701,  1.   ,  0.818],
        [ 0.327,  0.268,  0.552,  0.818,  1.   ]],

       [[ 1.   ,  0.265,  0.525,  0.569,  0.885],
        [ 0.265,  1.   ,  0.169,  0.473,  0.105],
        [ 0.525,  0.169,  1.   ,  0.8  ,  0.422],
        [ 0.569,  0.473,  0.8  ,  1.   ,  0.348],
        [ 0.885,  0.105,  0.422,  0.348,  1.   ]],

       [[ 1.   ,  0.052,  0.264,  0.253,  0.584],
        [ 0.052,  1.   ,  0.031,  0.335,  0.108],
        [ 0.264,  0.031,  1.   ,  0.506,  0.77 ],
        [ 0.253,  0.335,  0.506,  1.   ,  0.717],
        [ 0.584,  0.108,  0.77 ,  0.717,  1.   ]],

       [[ 1.   ,  0.091,  0.044,  0.006,  0.129],
        [ 0.091,  1.   ,  0.002,  0.025,  0.   ],
        [ 0.044,  0.002,  1.   ,  0.132,  0.249],
        [ 0.006,  0.025,  0.132,  1.   ,  0.002],
        [ 0.129,  0.   ,  0.249,  0.002,  1.

In [32]:
# Similarity on the zvec's (homophily latent space)
hawkes_test.dist_mat[0]

array([[ 1.   ,  0.734,  0.769,  0.638,  0.327],
       [ 0.734,  1.   ,  0.398,  0.611,  0.268],
       [ 0.769,  0.398,  1.   ,  0.701,  0.552],
       [ 0.638,  0.611,  0.701,  1.   ,  0.818],
       [ 0.327,  0.268,  0.552,  0.818,  1.   ]])

In [33]:
# Similarity on the xvec's (B reciprocal latent space)
hawkes_test.dist_mat[1:]

array([[[ 1.   ,  0.265,  0.525,  0.569,  0.885],
        [ 0.265,  1.   ,  0.169,  0.473,  0.105],
        [ 0.525,  0.169,  1.   ,  0.8  ,  0.422],
        [ 0.569,  0.473,  0.8  ,  1.   ,  0.348],
        [ 0.885,  0.105,  0.422,  0.348,  1.   ]],

       [[ 1.   ,  0.052,  0.264,  0.253,  0.584],
        [ 0.052,  1.   ,  0.031,  0.335,  0.108],
        [ 0.264,  0.031,  1.   ,  0.506,  0.77 ],
        [ 0.253,  0.335,  0.506,  1.   ,  0.717],
        [ 0.584,  0.108,  0.77 ,  0.717,  1.   ]],

       [[ 1.   ,  0.091,  0.044,  0.006,  0.129],
        [ 0.091,  1.   ,  0.002,  0.025,  0.   ],
        [ 0.044,  0.002,  1.   ,  0.132,  0.249],
        [ 0.006,  0.025,  0.132,  1.   ,  0.002],
        [ 0.129,  0.   ,  0.249,  0.002,  1.   ]],

       [[ 1.   ,  0.544,  0.048,  0.02 ,  0.192],
        [ 0.544,  1.   ,  0.01 ,  0.03 ,  0.017],
        [ 0.048,  0.01 ,  1.   ,  0.193,  0.214],
        [ 0.02 ,  0.03 ,  0.193,  1.   ,  0.006],
        [ 0.192,  0.017,  0.214,  0.006,  1.

In [30]:
hawkes_test.dist_mat
# See embedding.py and hawkes_embedding.dist_mat
plot_dist_heatmap(hawkes)  # Plots saved to directory. 

array([[[ 1.   ,  0.734,  0.769,  0.638,  0.327],
        [ 0.734,  1.   ,  0.398,  0.611,  0.268],
        [ 0.769,  0.398,  1.   ,  0.701,  0.552],
        [ 0.638,  0.611,  0.701,  1.   ,  0.818],
        [ 0.327,  0.268,  0.552,  0.818,  1.   ]],

       [[ 1.   ,  0.265,  0.525,  0.569,  0.885],
        [ 0.265,  1.   ,  0.169,  0.473,  0.105],
        [ 0.525,  0.169,  1.   ,  0.8  ,  0.422],
        [ 0.569,  0.473,  0.8  ,  1.   ,  0.348],
        [ 0.885,  0.105,  0.422,  0.348,  1.   ]],

       [[ 1.   ,  0.052,  0.264,  0.253,  0.584],
        [ 0.052,  1.   ,  0.031,  0.335,  0.108],
        [ 0.264,  0.031,  1.   ,  0.506,  0.77 ],
        [ 0.253,  0.335,  0.506,  1.   ,  0.717],
        [ 0.584,  0.108,  0.77 ,  0.717,  1.   ]],

       [[ 1.   ,  0.091,  0.044,  0.006,  0.129],
        [ 0.091,  1.   ,  0.002,  0.025,  0.   ],
        [ 0.044,  0.002,  1.   ,  0.132,  0.249],
        [ 0.006,  0.025,  0.132,  1.   ,  0.002],
        [ 0.129,  0.   ,  0.249,  0.002,  1.

In [35]:
hawkes_test.zvec
hawkes_test.xvec

array([[ 0.413,  0.127],
       [ 0.114,  0.595],
       [ 0.243, -0.356],
       [-0.243, -0.01 ],
       [-0.528, -0.356]])

array([[[-0.186,  0.337],
        [-0.324,  0.75 ],
        [ 0.146,  0.97 ],
        [ 0.346,  0.727]],

       [[ 0.914, -0.008],
        [ 1.168, -0.102],
        [ 1.517,  0.249],
        [ 1.078,  0.458]],

       [[-0.342, -0.451],
        [-0.679, -0.348],
        [-0.813, -0.516],
        [-0.798, -0.59 ]],

       [[ 0.119, -0.349],
        [ 0.146, -0.324],
        [ 0.376, -1.297],
        [ 0.303, -1.248]],

       [[-0.511,  0.463],
        [-0.319,  0.016],
        [-1.232,  0.586],
        [-0.936,  0.645]]])

In [37]:
# Latent features for the first node:
hawkes_test.zvec[0]
hawkes_test.xvec[0]

array([ 0.413,  0.127])

array([[-0.186,  0.337],
       [-0.324,  0.75 ],
       [ 0.146,  0.97 ],
       [ 0.346,  0.727]])

Computing the AUC and plotting the ROC curves involve splitting the dataset into training and test events; see embedding.py and hawkes_test.py for details.